In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import keras

2023-10-30 10:10:35.865955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 10:10:35.865982: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 10:10:35.866007: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 10:10:35.872099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 10:10:36.624215: W tensorflow/compiler/

In [2]:
df = pd.read_csv('concrete_data.csv')

In [3]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash               float64
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                     int64
Strength              float64
dtype: object

In [5]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
df = (df-df.mean())/df.std()

In [7]:
df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597,2.644123
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597,1.560663
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340,0.266498
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221,0.313188
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069,0.507732
...,...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597,0.506535
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597,-0.277627
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597,-0.725377
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597,-0.182450


In [8]:
### Split the dataframe

X = df.drop(columns=['Strength'])
y = df['Strength']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.3)

In [9]:
def regression_model():
    # create model
    model = Sequential()
    model.add(keras.Input(shape=(8,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [10]:
model = regression_model()

2023-10-30 10:10:37.504627: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
### Training the model 50 times. 

results_mse = []


for reg in range(1, 51):
    model.fit(X_train, y_train, epochs=100, verbose=2)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results_mse.append(mse)

Epoch 1/100
23/23 - 0s - loss: 1.0077 - 339ms/epoch - 15ms/step
Epoch 2/100
23/23 - 0s - loss: 0.8650 - 20ms/epoch - 870us/step
Epoch 3/100
23/23 - 0s - loss: 0.7667 - 20ms/epoch - 850us/step
Epoch 4/100
23/23 - 0s - loss: 0.6917 - 20ms/epoch - 849us/step
Epoch 5/100
23/23 - 0s - loss: 0.6400 - 22ms/epoch - 958us/step
Epoch 6/100
23/23 - 0s - loss: 0.5973 - 20ms/epoch - 862us/step
Epoch 7/100
23/23 - 0s - loss: 0.5654 - 18ms/epoch - 803us/step
Epoch 8/100
23/23 - 0s - loss: 0.5381 - 20ms/epoch - 870us/step
Epoch 9/100
23/23 - 0s - loss: 0.5171 - 20ms/epoch - 861us/step
Epoch 10/100
23/23 - 0s - loss: 0.4981 - 19ms/epoch - 814us/step
Epoch 11/100
23/23 - 0s - loss: 0.4820 - 19ms/epoch - 831us/step
Epoch 12/100
23/23 - 0s - loss: 0.4682 - 20ms/epoch - 867us/step
Epoch 13/100
23/23 - 0s - loss: 0.4583 - 20ms/epoch - 866us/step
Epoch 14/100
23/23 - 0s - loss: 0.4474 - 19ms/epoch - 809us/step
Epoch 15/100
23/23 - 0s - loss: 0.4363 - 19ms/epoch - 832us/step
Epoch 16/100
23/23 - 0s - loss: 0.

Epoch 27/100
23/23 - 0s - loss: 0.1282 - 23ms/epoch - 991us/step
Epoch 28/100
23/23 - 0s - loss: 0.1275 - 22ms/epoch - 976us/step
Epoch 29/100
23/23 - 0s - loss: 0.1274 - 21ms/epoch - 895us/step
Epoch 30/100
23/23 - 0s - loss: 0.1272 - 23ms/epoch - 984us/step
Epoch 31/100
23/23 - 0s - loss: 0.1270 - 23ms/epoch - 993us/step
Epoch 32/100
23/23 - 0s - loss: 0.1267 - 22ms/epoch - 960us/step
Epoch 33/100
23/23 - 0s - loss: 0.1266 - 21ms/epoch - 934us/step
Epoch 34/100
23/23 - 0s - loss: 0.1262 - 22ms/epoch - 948us/step
Epoch 35/100
23/23 - 0s - loss: 0.1259 - 24ms/epoch - 1ms/step
Epoch 36/100
23/23 - 0s - loss: 0.1257 - 22ms/epoch - 955us/step
Epoch 37/100
23/23 - 0s - loss: 0.1253 - 20ms/epoch - 865us/step
Epoch 38/100
23/23 - 0s - loss: 0.1253 - 19ms/epoch - 833us/step
Epoch 39/100
23/23 - 0s - loss: 0.1253 - 22ms/epoch - 940us/step
Epoch 40/100
23/23 - 0s - loss: 0.1251 - 21ms/epoch - 922us/step
Epoch 41/100
23/23 - 0s - loss: 0.1252 - 23ms/epoch - 979us/step
Epoch 42/100
23/23 - 0s - l

Epoch 53/100
23/23 - 0s - loss: 0.1113 - 20ms/epoch - 856us/step
Epoch 54/100
23/23 - 0s - loss: 0.1111 - 20ms/epoch - 851us/step
Epoch 55/100
23/23 - 0s - loss: 0.1117 - 21ms/epoch - 925us/step
Epoch 56/100
23/23 - 0s - loss: 0.1117 - 21ms/epoch - 916us/step
Epoch 57/100
23/23 - 0s - loss: 0.1108 - 19ms/epoch - 808us/step
Epoch 58/100
23/23 - 0s - loss: 0.1113 - 20ms/epoch - 854us/step
Epoch 59/100
23/23 - 0s - loss: 0.1113 - 18ms/epoch - 774us/step
Epoch 60/100
23/23 - 0s - loss: 0.1109 - 19ms/epoch - 824us/step
Epoch 61/100
23/23 - 0s - loss: 0.1110 - 18ms/epoch - 799us/step
Epoch 62/100
23/23 - 0s - loss: 0.1113 - 18ms/epoch - 787us/step
Epoch 63/100
23/23 - 0s - loss: 0.1113 - 18ms/epoch - 787us/step
Epoch 64/100
23/23 - 0s - loss: 0.1106 - 18ms/epoch - 789us/step
Epoch 65/100
23/23 - 0s - loss: 0.1107 - 19ms/epoch - 838us/step
Epoch 66/100
23/23 - 0s - loss: 0.1110 - 19ms/epoch - 814us/step
Epoch 67/100
23/23 - 0s - loss: 0.1110 - 19ms/epoch - 824us/step
Epoch 68/100
23/23 - 0s -

Epoch 79/100
23/23 - 0s - loss: 0.1069 - 20ms/epoch - 850us/step
Epoch 80/100
23/23 - 0s - loss: 0.1063 - 18ms/epoch - 790us/step
Epoch 81/100
23/23 - 0s - loss: 0.1060 - 19ms/epoch - 825us/step
Epoch 82/100
23/23 - 0s - loss: 0.1063 - 20ms/epoch - 869us/step
Epoch 83/100
23/23 - 0s - loss: 0.1062 - 18ms/epoch - 795us/step
Epoch 84/100
23/23 - 0s - loss: 0.1059 - 18ms/epoch - 795us/step
Epoch 85/100
23/23 - 0s - loss: 0.1066 - 18ms/epoch - 767us/step
Epoch 86/100
23/23 - 0s - loss: 0.1061 - 17ms/epoch - 755us/step
Epoch 87/100
23/23 - 0s - loss: 0.1060 - 17ms/epoch - 725us/step
Epoch 88/100
23/23 - 0s - loss: 0.1062 - 18ms/epoch - 765us/step
Epoch 89/100
23/23 - 0s - loss: 0.1064 - 18ms/epoch - 793us/step
Epoch 90/100
23/23 - 0s - loss: 0.1059 - 18ms/epoch - 791us/step
Epoch 91/100
23/23 - 0s - loss: 0.1069 - 18ms/epoch - 803us/step
Epoch 92/100
23/23 - 0s - loss: 0.1072 - 17ms/epoch - 760us/step
Epoch 93/100
23/23 - 0s - loss: 0.1072 - 22ms/epoch - 942us/step
Epoch 94/100
23/23 - 0s -

Epoch 4/100
23/23 - 0s - loss: 0.1034 - 19ms/epoch - 814us/step
Epoch 5/100
23/23 - 0s - loss: 0.1031 - 18ms/epoch - 802us/step
Epoch 6/100
23/23 - 0s - loss: 0.1026 - 17ms/epoch - 759us/step
Epoch 7/100
23/23 - 0s - loss: 0.1031 - 18ms/epoch - 762us/step
Epoch 8/100
23/23 - 0s - loss: 0.1038 - 17ms/epoch - 757us/step
Epoch 9/100
23/23 - 0s - loss: 0.1033 - 19ms/epoch - 827us/step
Epoch 10/100
23/23 - 0s - loss: 0.1033 - 18ms/epoch - 774us/step
Epoch 11/100
23/23 - 0s - loss: 0.1048 - 18ms/epoch - 801us/step
Epoch 12/100
23/23 - 0s - loss: 0.1024 - 18ms/epoch - 782us/step
Epoch 13/100
23/23 - 0s - loss: 0.1025 - 18ms/epoch - 794us/step
Epoch 14/100
23/23 - 0s - loss: 0.1026 - 18ms/epoch - 762us/step
Epoch 15/100
23/23 - 0s - loss: 0.1027 - 17ms/epoch - 747us/step
Epoch 16/100
23/23 - 0s - loss: 0.1029 - 18ms/epoch - 774us/step
Epoch 17/100
23/23 - 0s - loss: 0.1030 - 18ms/epoch - 769us/step
Epoch 18/100
23/23 - 0s - loss: 0.1030 - 17ms/epoch - 760us/step
Epoch 19/100
23/23 - 0s - loss:

...

Epoch 2/100
23/23 - 0s - loss: 0.0901 - 17ms/epoch - 751us/step
Epoch 3/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 783us/step
Epoch 4/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 795us/step
Epoch 5/100
23/23 - 0s - loss: 0.0905 - 17ms/epoch - 744us/step
Epoch 6/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 793us/step
Epoch 7/100
23/23 - 0s - loss: 0.0900 - 19ms/epoch - 829us/step
Epoch 8/100
23/23 - 0s - loss: 0.0897 - 17ms/epoch - 728us/step
Epoch 9/100
23/23 - 0s - loss: 0.0898 - 17ms/epoch - 739us/step
Epoch 10/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 795us/step
Epoch 11/100
23/23 - 0s - loss: 0.0901 - 17ms/epoch - 751us/step
Epoch 12/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 778us/step
Epoch 13/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 800us/step
Epoch 14/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 766us/step
Epoch 15/100
23/23 - 0s - loss: 0.0899 - 17ms/epoch - 739us/step
Epoch 16/100
23/23 - 0s - loss: 0.0907 - 19ms/epoch - 806us/step
Epoch 17/100
23/23 - 0s - loss: 0

Epoch 28/100
23/23 - 0s - loss: 0.0901 - 18ms/epoch - 788us/step
Epoch 29/100
23/23 - 0s - loss: 0.0902 - 18ms/epoch - 798us/step
Epoch 30/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 794us/step
Epoch 31/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 786us/step
Epoch 32/100
23/23 - 0s - loss: 0.0902 - 17ms/epoch - 761us/step
Epoch 33/100
23/23 - 0s - loss: 0.0907 - 19ms/epoch - 836us/step
Epoch 34/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 782us/step
Epoch 35/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 798us/step
Epoch 36/100
23/23 - 0s - loss: 0.0897 - 19ms/epoch - 809us/step
Epoch 37/100
23/23 - 0s - loss: 0.0899 - 19ms/epoch - 814us/step
Epoch 38/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 785us/step
Epoch 39/100
23/23 - 0s - loss: 0.0906 - 18ms/epoch - 785us/step
Epoch 40/100
23/23 - 0s - loss: 0.0902 - 19ms/epoch - 812us/step
Epoch 41/100
23/23 - 0s - loss: 0.0902 - 19ms/epoch - 810us/step
Epoch 42/100
23/23 - 0s - loss: 0.0901 - 18ms/epoch - 795us/step
Epoch 43/100
23/23 - 0s -

Epoch 54/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 791us/step
Epoch 55/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 782us/step
Epoch 56/100
23/23 - 0s - loss: 0.0896 - 18ms/epoch - 782us/step
Epoch 57/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 796us/step
Epoch 58/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 780us/step
Epoch 59/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 781us/step
Epoch 60/100
23/23 - 0s - loss: 0.0903 - 18ms/epoch - 768us/step
Epoch 61/100
23/23 - 0s - loss: 0.0902 - 19ms/epoch - 830us/step
Epoch 62/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 786us/step
Epoch 63/100
23/23 - 0s - loss: 0.0907 - 18ms/epoch - 768us/step
Epoch 64/100
23/23 - 0s - loss: 0.0898 - 17ms/epoch - 725us/step
Epoch 65/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 770us/step
Epoch 66/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 774us/step
Epoch 67/100
23/23 - 0s - loss: 0.0901 - 18ms/epoch - 804us/step
Epoch 68/100
23/23 - 0s - loss: 0.0905 - 17ms/epoch - 749us/step
Epoch 69/100
23/23 - 0s -

Epoch 80/100
23/23 - 0s - loss: 0.0899 - 19ms/epoch - 820us/step
Epoch 81/100
23/23 - 0s - loss: 0.0900 - 19ms/epoch - 819us/step
Epoch 82/100
23/23 - 0s - loss: 0.0898 - 19ms/epoch - 814us/step
Epoch 83/100
23/23 - 0s - loss: 0.0894 - 18ms/epoch - 798us/step
Epoch 84/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 795us/step
Epoch 85/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 775us/step
Epoch 86/100
23/23 - 0s - loss: 0.0899 - 18ms/epoch - 776us/step
Epoch 87/100
23/23 - 0s - loss: 0.0900 - 18ms/epoch - 793us/step
Epoch 88/100
23/23 - 0s - loss: 0.0897 - 19ms/epoch - 811us/step
Epoch 89/100
23/23 - 0s - loss: 0.0901 - 22ms/epoch - 949us/step
Epoch 90/100
23/23 - 0s - loss: 0.0897 - 18ms/epoch - 803us/step
Epoch 91/100
23/23 - 0s - loss: 0.0894 - 19ms/epoch - 825us/step
Epoch 92/100
23/23 - 0s - loss: 0.0902 - 19ms/epoch - 806us/step
Epoch 93/100
23/23 - 0s - loss: 0.0902 - 19ms/epoch - 817us/step
Epoch 94/100
23/23 - 0s - loss: 0.0898 - 18ms/epoch - 799us/step
Epoch 95/100
23/23 - 0s -

Epoch 5/100
23/23 - 0s - loss: 0.0897 - 20ms/epoch - 848us/step
Epoch 6/100
23/23 - 0s - loss: 0.0904 - 22ms/epoch - 945us/step
Epoch 7/100
23/23 - 0s - loss: 0.0897 - 21ms/epoch - 894us/step
Epoch 8/100
23/23 - 0s - loss: 0.0900 - 21ms/epoch - 893us/step
Epoch 9/100
23/23 - 0s - loss: 0.0897 - 21ms/epoch - 904us/step
Epoch 10/100
23/23 - 0s - loss: 0.0897 - 22ms/epoch - 945us/step
Epoch 11/100
23/23 - 0s - loss: 0.0897 - 20ms/epoch - 890us/step
Epoch 12/100
23/23 - 0s - loss: 0.0896 - 21ms/epoch - 903us/step
Epoch 13/100
23/23 - 0s - loss: 0.0897 - 21ms/epoch - 906us/step
Epoch 14/100
23/23 - 0s - loss: 0.0899 - 23ms/epoch - 1ms/step
Epoch 15/100
23/23 - 0s - loss: 0.0901 - 24ms/epoch - 1ms/step
Epoch 16/100
23/23 - 0s - loss: 0.0894 - 20ms/epoch - 889us/step
Epoch 17/100
23/23 - 0s - loss: 0.0893 - 21ms/epoch - 910us/step
Epoch 18/100
23/23 - 0s - loss: 0.0896 - 20ms/epoch - 857us/step
Epoch 19/100
23/23 - 0s - loss: 0.0895 - 21ms/epoch - 926us/step
Epoch 20/100
23/23 - 0s - loss: 0.

In [12]:
results_mse

[0.1765703147375805,
 0.15238466415146618,
 0.1457941276150316,
 0.1465556200305673,
 0.14452881156982803,
 0.14215088052770136,
 0.14179447103176113,
 0.13945374694209917,
 0.13990867255763634,
 0.13891605487254788,
 0.13716196264721228,
 0.1373273238319626,
 0.1374221774415216,
 0.13836007584856597,
 0.13898219950569562,
 0.1385771692657182,
 0.13962609123995753,
 0.13897306076071098,
 0.13849718843886186,
 0.1385822130586195,
 0.13841596515973587,
 0.1396280483781705,
 0.13967876701264845,
 0.1399706631287237,
 0.14001367790254415,
 0.13947538480148064,
 0.14158044878971923,
 0.13978198024681693,
 0.14009803784850391,
 0.1392125485200998,
 0.13880715414163639,
 0.13990845917916445,
 0.14004852757724942,
 0.14074366479566794,
 0.14140715732477208,
 0.14121549255149876,
 0.14063615932645682,
 0.1406305885726738,
 0.14076384072749024,
 0.1403113900824146,
 0.14014168631900978,
 0.14124470169202738,
 0.14058674906597626,
 0.1401243186302725,
 0.14033434500973674,
 0.14107006403382272,
 

In [13]:
mse_df = pd.DataFrame(results_mse)

In [14]:
mse_df.head()

,0
0,0.176570
1,0.152385
2,0.145794
3,0.146556
4,0.144529


In [15]:
mse_df.rename(columns={0:'Mean Squared Error'}, inplace=True)

In [16]:
### Mean of the Mean Squared errors.

mse_df['Mean Squared Error'].mean()

0.1411001194118885

In [17]:
### Standard Deviation of the Mean Squared errors.

mse_df['Mean Squared Error'].std()

0.0056945211164245385

C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?